In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



from scipy.stats import mannwhitneyu


%matplotlib inline
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

        

In [ ]:
# Уровень значимости alpha
ALPHA = .05

In [ ]:
df_train = pd.read_csv('/kaggle/input/gbcreditdefault/train.csv')
df_test = pd.read_csv('/kaggle/input/gbcreditdefault/test.csv')

In [ ]:
df_train

In [ ]:
df_train.info()

In [ ]:
BASE_FEATURES = ['Home Ownership', # домовладение
    'Annual Income', # годовой доход
    'Years in current job', # количество лет на текущем месте работы
    'Tax Liens', # налоговые обременения
    'Number of Open Accounts', # количество открытых счетов
    'Years of Credit History', # количество лет кредитной истории
    'Maximum Open Credit', # наибольший открытый кредит
    'Number of Credit Problems', # количество проблем с кредитом
    'Months since last delinquent', # количество месяцев с последней просрочки платежа
    'Bankruptcies', # банкротства
    'Purpose', # цель кредита
    'Term', # срок кредита
    'Current Loan Amount', # текущая сумма кредита
    'Current Credit Balance', # текущий кредитный баланс
    'Monthly Debt', # ежемесячный долг
    'Credit Score'#  баллы кредитного рейтинга
]
TARGET = 'Credit Default' # факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)

In [ ]:
# Проверка наличия у признака меньше 10 уникальных значений - возможные категориальные признаки
index = 0
print('%-33s' % '     FEATURE',
    '%+4s' % 'STR',
    '%+10s' % 'UNIQUE<10',
    '%+10s' % 'UNIQUES')
for feature, is_str, nunique in list(zip(
    df_train.columns[:-1],
    df_train.dtypes[:-1] == object,
    df_train.nunique())):
    index += 1
    print(f"{index:>4} " +
        f"{feature:<{len(np.max(df_train.columns)) + 5}}" +
        f"{is_str:>4}" +
        f"{nunique < 10:>10}" +
        f"{nunique:>10}")

In [ ]:
# Формируем список категориальных признаков
CAT_FEATURES = [feature for feature in df_train.columns[:-1]
    if df_train[feature].nunique() < 10 or df_train[feature].dtypes == object]
CAT_FEATURES

In [ ]:
# домовладение
df_train['Home Ownership'].value_counts()

In [ ]:
# количество лет на текущем месте работы
df_train['Years in current job'].value_counts()

In [ ]:
# налоговые обременения
df_train['Tax Liens'].value_counts()

In [ ]:
# количество проблем с кредитом
df_train['Number of Credit Problems'].value_counts()

In [ ]:
# банкротства
df_train['Bankruptcies'].value_counts()

In [ ]:
# цель кредита
df_train['Purpose'].value_counts()

In [ ]:
# срок кредита
df_train['Term'].value_counts()

In [ ]:
# Формируем список вещественных признаков
NUM_FEATURES = list(set(df_train.columns[:-1]) - set(CAT_FEATURES))
NUM_FEATURES

In [ ]:
# Проверка корреляции между признаками:
df_train.corr().style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
# Оценка схожести вещественных признаков в обучающей и тестовой выборках
p_values = dict()
for df in [df_train]:
    for feature in NUM_FEATURES:
        plt.figure(figsize=(16, 8))
        stat, pvalue = mannwhitneyu(df_train[feature], df_test[feature])
        p_values[feature] = pvalue
        if pvalue < ALPHA:
            print(f'Признак: {feature}, p-value = {pvalue} = > ', end='')
            print('Средние в выборках НЕ эквивалентны!')
        sns.kdeplot(df_train[feature], shade=True, label='train', color='g')
        sns.kdeplot(df_test[feature], shade=True, label='test', color='r')
        plt.xlabel(TARGET)
        plt.ylabel(feature)
        plt.title(f'Numbers feature: "{feature}" in train and test\n'+
                  f'Mann-Whitneyu: p-value = {pvalue} = > Equal: {pvalue > ALPHA}')
        plt.show()

In [ ]:
p_values

In [ ]:
# Оценка схожести вещественных признаков в разрезе целевой переменной при 'target'=0 и 'target'=1:
for df in [df_train]:
    for feature in NUM_FEATURES:
        plt.figure(figsize=(16, 8))
        sns.kdeplot(df_train[df_train[TARGET] == 0][feature], shade=True, label='Нет просрочки', color='g')
        sns.kdeplot(df_train[df_train[TARGET] == 1][feature], shade=True, label='Есть просрочка', color='r')
#         plt.xlim(500, 800)
        plt.xlabel(feature)
        plt.ylabel(feature)
        plt.title(f'{feature}')
        plt.show()


Согласно оценке схожести вещественного признака 'Credit Score', как одного из наиболее значимых, что также подтверждает таблица корреляции, его значения при значении целевой переменной "нет просрочки" ('Credit Default'=0) лежат в диапазоне [590..751]

In [ ]:
# Оценка 'Credit Score'
df_train.groupby(['Credit Default'])['Credit Score', 'Credit Default'].agg({
    'Credit Score': ['min', 'max', 'mean', 'median'],
    'Credit Default': 'count'
}).reset_index()

In [ ]:
# Оценка 'Annual Income'
df_train.groupby(['Credit Default'])['Annual Income', 'Credit Default'].agg({
    'Annual Income': ['min', 'max', 'mean', 'median'],
    'Credit Default': 'count'
}).reset_index()

In [ ]:
# Оценка схожести и выбросов вещественных признаков в разрезе целевой переменной при 'target'=0 и 'target'=1:
for df in [df_train]:
    for feature in BASE_FEATURES:
        plt.figure(figsize=(16, 8))
        sns.boxplot(TARGET, feature, data=df_train, palette='rainbow')
        plt.xlabel(TARGET)
        plt.ylabel(feature)
        plt.title(f'{feature}')
        plt.show()

In [ ]:
# Оценка схожести категориальных признаков в обучающей и тестовой выборках

num_feature = 'Credit Score'

for df in [df_train]:
    for feature in CAT_FEATURES:
        plt.figure(figsize=(16, 8))
        sns.pointplot(x=feature, y=num_feature, data=df_train, capsize=.1, label='train', color='g')
        sns.pointplot(x=feature, y=num_feature, data=df_test, capsize=.1, label='test', color='r')
        plt.legend()
        plt.title(feature)
        plt.show()

--------------------------------

In [ ]:
# Проверка наличия уникальных значений меньше 10 - возможные 
# Формируем список категориальных признаков
NUM_FEATURES = [feature for feature in BASE_FEATURES if isinstance(df_train[feature][:], (float, int))]
NUM_FEATURES

for feature in df_train.columns[:-1]:
#     print(f"{feature:<{len(np.max(df_train.columns)) + 5}} {df_train[feature].nunique():>6} " +
#           f"{df_train[feature].nunique() < 10 or isinstance(df_train[feature][:], str)}")
    print(f"{feature:<{len(np.max(df_train.columns)) + 5}} {df_train[feature].nunique():>6} " +
          f"{isinstance(df_train[feature][0], str)}")

In [ ]:
df_train.Bankruptcies.value_counts()

In [ ]:
NUM_FEATURES = ['Annual Income', # годовой доход
    'Tax Liens', # налоговые обременения
    'Number of Open Accounts', # количество открытых счетов
    'Years of Credit History', # количество лет кредитной истории
    'Maximum Open Credit', # наибольший открытый кредит
    'Number of Credit Problems', # количество проблем с кредитом
    'Months since last delinquent', # количество месяцев с последней просрочки платежа
    'Bankruptcies', # банкротства
    'Current Loan Amount', # текущая сумма кредита
    'Current Credit Balance', # текущий кредитный баланс
    'Monthly Debt', # ежемесячный долг
    'Credit Score'#  баллы кредитного рейтинга
]
CAT_FEATURES = ['Home Ownership', # домовладение
    'Years in current job', # количество лет на текущем месте работы
    'Purpose', # цель кредита
    'Term' # срок кредита
]